# Metadata investigation

## French key name to English key name

In [ ]:
# Create the translation mapping
french_to_english = {
    "id": "id",
    "titre": "title",
    "url": "source_url",
    "domaine": "domain",
    "prononciation": "date_pronounced",
    "intervenants": "speakers",
    "nom": "speaker_name",
    "qualite": "speaker_role",
    "qualite_long": "speaker_role_extended",
    "auteur_moral": "corporate_author",
    "circonstance": "circumstance",
    "type_emetteur": "issuer_type",
    "type_document": "document_type",
    "thematiques": "themes",
    "descripteurs": "tags",
    "mise_en_ligne": "date_published",
    "mise_a_jour": "date_updated"
}

## Load the Metadata from json file to data frame

In [19]:
import pandas as pd
import json

# 1. Load the data
with open('dataset/vp_discours.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)
print(f"Total records loaded: {len(raw_data)}")

# 2. Basic DataFrame creation
df = pd.DataFrame(raw_data)

display(df.head())

Total records loaded: 152089


,id,titre,url,domaine,prononciation,intervenants,auteur_moral,circonstance,type_emetteur,type_document,type_media,media,resume,thematiques,descripteurs,mise_en_ligne,mise_a_jour
0,256002092,Conseil des ministres du 22 décembre 2025. Loi...,https://www.vie-publique.fr/discours/301466-co...,Conseil des ministres,2025-12-22,"[{'nom': 'Roland Lescure', 'qualite': 'Ministr...",[Secrétariat général du Gouvernement],None,Conseil des ministres,Communiqué,None,None,None,[Budget de l'État],"[Projet de loi, Budget 2026, Loi organique, Lo...",2025-12-23,2025-12-23
1,253002081,"Interview de Mme Maud Bregeon, ministre délégu...",https://www.vie-publique.fr/discours/301442-ma...,Déclaration,2025-12-19,"[{'nom': 'Maud Bregeon', 'qualite': 'Ministre ...",[],None,Gouvernement,Interview,Radio,RTL,None,[Commerce international],"[Politique agricole, Épizootie, Exploitation a...",2025-12-19,2025-12-23
2,253002082,"Interview de M. Serge Papin, ministre des Peti...",https://www.vie-publique.fr/discours/301443-se...,Déclaration,2025-12-19,"[{'nom': 'Serge Papin', 'qualite': 'Ministre d...",[],None,Gouvernement,Interview,Télévision,France 2,None,[Politique économique],"[UE - MERCOSUR, Industrie du jouet, Commerce, ...",2025-12-19,2025-12-19
3,253002083,"Interview de Mme Amélie de Montchalin, ministr...",https://www.vie-publique.fr/discours/301444-am...,Déclaration,2025-12-19,"[{'nom': 'Amélie de Montchalin', 'qualite': 'M...",[],None,Gouvernement,Interview,Télévision,TF1,None,[Budget de l'État],"[Budget 2026, Parlement, Politique économique]",2025-12-19,2025-12-19
4,252002085,"Communiqué de la Présidence de la République, ...",https://www.vie-publique.fr/discours/301446-pr...,Communiqué,2025-12-18,"[{'nom': None, 'qualite': None, 'qualite_long'...",[Présidence de la République],Réunion du nouveau groupe de pilotage nucléair...,Présidence de la République,Communiqué,None,None,None,[Politique de la défense],"[Arme nucléaire, Coopération, France - Grande ...",2025-12-19,2025-12-19


In [ ]:
## Translate column names from French to English and Flatten nested structures

In [ ]:
# 3. Investigation: Handle the nested 'intervenants' column
# Since one document can have multiple speakers, we 'explode' the list
# Explode the speakers
df_flat = df.explode('intervenants')

# Normalize and rename the speaker-specific keys
speakers = pd.json_normalize(df_flat['intervenants']).rename(columns=french_to_english)

# Combine and rename the main keys
df_final = pd.concat([df_flat.drop('intervenants', axis=1).reset_index(drop=True), 
                     speakers.reset_index(drop=True)], axis=1)

df_final = df_final.rename(columns=french_to_english)

,id,title,source_url,domain,date_pronounced,corporate_author,circumstance,issuer_type,document_type,type_media,media,resume,themes,tags,date_published,date_updated,speaker_name,speaker_role,speaker_role_extended
0,256002092,Conseil des ministres du 22 décembre 2025. Loi...,https://www.vie-publique.fr/discours/301466-co...,Conseil des ministres,2025-12-22,[Secrétariat général du Gouvernement],None,Conseil des ministres,Communiqué,None,None,None,[Budget de l'État],"[Projet de loi, Budget 2026, Loi organique, Lo...",2025-12-23,2025-12-23,Roland Lescure,"Ministre de l'économie, des finances et de la ...",None
1,256002092,Conseil des ministres du 22 décembre 2025. Loi...,https://www.vie-publique.fr/discours/301466-co...,Conseil des ministres,2025-12-22,[Secrétariat général du Gouvernement],None,Conseil des ministres,Communiqué,None,None,None,[Budget de l'État],"[Projet de loi, Budget 2026, Loi organique, Lo...",2025-12-23,2025-12-23,Amélie de Montchalin,Ministre de l'action et des comptes publics,None
2,253002081,"Interview de Mme Maud Bregeon, ministre délégu...",https://www.vie-publique.fr/discours/301442-ma...,Déclaration,2025-12-19,[],None,Gouvernement,Interview,Radio,RTL,None,[Commerce international],"[Politique agricole, Épizootie, Exploitation a...",2025-12-19,2025-12-23,Maud Bregeon,"Ministre déléguée, porte-parole du Gouvernement",None
3,253002081,"Interview de Mme Maud Bregeon, ministre délégu...",https://www.vie-publique.fr/discours/301442-ma...,Déclaration,2025-12-19,[],None,Gouvernement,Interview,Radio,RTL,None,[Commerce international],"[Politique agricole, Épizootie, Exploitation a...",2025-12-19,2025-12-23,Thomas Sotto,Journaliste,None
4,253002082,"Interview de M. Serge Papin, ministre des Peti...",https://www.vie-publique.fr/discours/301443-se...,Déclaration,2025-12-19,[],None,Gouvernement,Interview,Télévision,France 2,None,[Politique économique],"[UE - MERCOSUR, Industrie du jouet, Commerce, ...",2025-12-19,2025-12-19,Serge Papin,"Ministre des Petites et moyennes entreprises, ...",None


In [ ]:
# 4. Display the results, view the clean, English column name dataset
print("### DataFrame Overview ###")
display(df_final.head())

print("\n### Data Types and Null Values ###")
display(df_final.info())

### DataFrame Overview ###


,id,title,source_url,domain,date_pronounced,corporate_author,circumstance,issuer_type,document_type,type_media,media,resume,themes,tags,date_published,date_updated,speaker_name,speaker_role,speaker_role_extended
0,256002092,Conseil des ministres du 22 décembre 2025. Loi...,https://www.vie-publique.fr/discours/301466-co...,Conseil des ministres,2025-12-22,[Secrétariat général du Gouvernement],None,Conseil des ministres,Communiqué,None,None,None,[Budget de l'État],"[Projet de loi, Budget 2026, Loi organique, Lo...",2025-12-23,2025-12-23,Roland Lescure,"Ministre de l'économie, des finances et de la ...",None
1,256002092,Conseil des ministres du 22 décembre 2025. Loi...,https://www.vie-publique.fr/discours/301466-co...,Conseil des ministres,2025-12-22,[Secrétariat général du Gouvernement],None,Conseil des ministres,Communiqué,None,None,None,[Budget de l'État],"[Projet de loi, Budget 2026, Loi organique, Lo...",2025-12-23,2025-12-23,Amélie de Montchalin,Ministre de l'action et des comptes publics,None
2,253002081,"Interview de Mme Maud Bregeon, ministre délégu...",https://www.vie-publique.fr/discours/301442-ma...,Déclaration,2025-12-19,[],None,Gouvernement,Interview,Radio,RTL,None,[Commerce international],"[Politique agricole, Épizootie, Exploitation a...",2025-12-19,2025-12-23,Maud Bregeon,"Ministre déléguée, porte-parole du Gouvernement",None
3,253002081,"Interview de Mme Maud Bregeon, ministre délégu...",https://www.vie-publique.fr/discours/301442-ma...,Déclaration,2025-12-19,[],None,Gouvernement,Interview,Radio,RTL,None,[Commerce international],"[Politique agricole, Épizootie, Exploitation a...",2025-12-19,2025-12-23,Thomas Sotto,Journaliste,None
4,253002082,"Interview de M. Serge Papin, ministre des Peti...",https://www.vie-publique.fr/discours/301443-se...,Déclaration,2025-12-19,[],None,Gouvernement,Interview,Télévision,France 2,None,[Politique économique],"[UE - MERCOSUR, Industrie du jouet, Commerce, ...",2025-12-19,2025-12-19,Serge Papin,"Ministre des Petites et moyennes entreprises, ...",None



### Data Types and Null Values ###
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206770 entries, 0 to 206769
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   id                     206485 non-null  object
 1   title                  206770 non-null  object
 2   source_url             206770 non-null  object
 3   domain                 206143 non-null  object
 4   date_pronounced        206770 non-null  object
 5   corporate_author       206770 non-null  object
 6   circumstance           80778 non-null   object
 7   issuer_type            194223 non-null  object
 8   document_type          206652 non-null  object
 9   type_media             36850 non-null   object
 10  media                  87408 non-null   object
 11  resume                 22480 non-null   object
 12  themes                 206770 non-null  object
 13  tags                   206770 non-null  object
 14  date_published  

None

In [ ]:
# 5. Feature Extraction: Count number of speakers
df_final['speaker_count'] = df_final.groupby('id')['speaker_name'].transform('count')
# Display the speaker count per document
display(df_final[['title', 'speaker_name', 'speaker_count']])

,title,speaker_name,speaker_count
0,Conseil des ministres du 22 décembre 2025. Loi...,Roland Lescure,2.0
1,Conseil des ministres du 22 décembre 2025. Loi...,Amélie de Montchalin,2.0
2,"Interview de Mme Maud Bregeon, ministre délégu...",Maud Bregeon,2.0
3,"Interview de Mme Maud Bregeon, ministre délégu...",Thomas Sotto,2.0
4,"Interview de M. Serge Papin, ministre des Peti...",Serge Papin,2.0
...,...,...,...
206765,Déclaration de politique générale de M. Pierre...,Pierre Messmer,1.0
206766,Déclaration de politique générale de M. Jacque...,Jacques Chaban-Delmas,1.0
206767,Déclaration de politique générale de M. Mauric...,Maurice Couve de Murville,1.0
206768,Déclaration de politique générale de M. George...,Georges Pompidou,1.0


In [26]:
# 6. Convert date strings to datetime objects for analysis
df_final['date_pronounced'] = pd.to_datetime(df_final['date_pronounced'])
df_final['date_published'] = pd.to_datetime(df_final['date_published'])

In [27]:
# Calculate processing lag (days between meeting and online publication)
df_final['publication_lag'] = (df_final['date_published'] - df_final['date_pronounced']).dt.days

print("\n### Key Metrics ###")
display(df_final[['title', 'speaker_name', 'publication_lag', 'speaker_count']])


### Key Metrics ###


,title,speaker_name,publication_lag,speaker_count
0,Conseil des ministres du 22 décembre 2025. Loi...,Roland Lescure,1,2.0
1,Conseil des ministres du 22 décembre 2025. Loi...,Amélie de Montchalin,1,2.0
2,"Interview de Mme Maud Bregeon, ministre délégu...",Maud Bregeon,0,2.0
3,"Interview de Mme Maud Bregeon, ministre délégu...",Thomas Sotto,0,2.0
4,"Interview de M. Serge Papin, ministre des Peti...",Serge Papin,0,2.0
...,...,...,...,...
206765,Déclaration de politique générale de M. Pierre...,Pierre Messmer,17076,1.0
206766,Déclaration de politique générale de M. Jacque...,Jacques Chaban-Delmas,18189,1.0
206767,Déclaration de politique générale de M. Mauric...,Maurice Couve de Murville,18615,1.0
206768,Déclaration de politique générale de M. George...,Georges Pompidou,20889,1.0


In [ ]:
# Optional: Save the cleaned DataFrame to a new CSV
# df_final.to_csv("french_gov_data_english.csv", index=False, encoding='utf-8-sig')

In [ ]:
# Filer out certain time of json files if needed
df_partial = df_final[df_final['date_published']].dt.year >= 2010

